# MODEL DEPLOYMENT

## Churn Class

In [6]:
class Churn( object ):
    
    def __init__( self ):
            import pickle
            self.home_path='/Users/brunoschirmer/repos/churn'
            self.credit_score_scaler  = pickle.load(open('/Users/brunoschirmer/Repos/churn/parameters/credit_score_scaler.pkl', 'rb'))
            self.age_scaler  =          pickle.load(open('/Users/brunoschirmer/Repos/churn/parameters/age_scaler.pkl', 'rb'))
            self.estimated_salary_scaler =  pickle.load(open('/Users/brunoschirmer/Repos/churn/parameters/estimated_salary_scaler.pkl', 'rb'))
            self.tenure_scaler =  pickle.load(open('/Users/brunoschirmer/Repos/churn/parameters/tenure_scaler.pkl', 'rb'))
            self.balance_scaler  = pickle.load(open('/Users/brunoschirmer/Repos/churn/parameters/balance_scaler.pkl', 'rb'))
            self.model = pickle.load(open('/Users/brunoschirmer/Repos/churn/src/model.pkl', 'rb'))
        

    def data_preparation( self, data ):
        
            #transform new columns
            cols_old = data.columns.to_list()
            snakecase = lambda x: inflection.underscore(x)
            cols_new = list(map(snakecase, cols_old))

            #rename
            data.columns = cols_new
            
            #drop columns
            data.drop(['customer_id','surname'], axis=1, inplace=True)

            # Gender Encoding
            data['gender'] = data['gender'].map({'Male': 0, 'Female': 1})

            #Geography encoding
            data = pd.get_dummies( data, prefix=['geography'], columns=['geography'] )

            #Credit Score
            data['credit_score'] = self.credit_score_scaler.transform(data[['credit_score']].values)

            #Age
            data['age'] = self.age_scaler.transform(data[['age']].values)

            #Estimated Salary
            data['estimated_salary'] = self.estimated_salary_scaler.transform(data[['estimated_salary']].values)

            #Tenure
            data['tenure'] = self.tenure_scalertransform(data[['tenure']].values)

            #Balance
            data['balance'] = self.balance_scaler.transform(data[['balance']].values)
                                                        
            return data                                            
            
    
    def get_prediction( self, model, original_data, data ):
        
            # prediction
            pred = model.predict( data )

            # join pred into the original data
            original_data['prediction'] = pred

            return original_data.to_json( orient='records', date_format='iso' )

## API Handler

In [7]:
import pickle
import pandas as pd
import numpy as np
from churn.Churn import Churn
from flask import Flask, request, Response

# loading model
model = pickle.load(open('/Users/brunoschirmer/Repos/churn/src/model.pkl', 'rb'))

# initialize API
app = Flask(__name__)

@app.route('/churn/predict', methods=['POST'])
def churn_predict():
    test_json = request.get_json()
   
    if test_json: # there is data
        if isinstance(test_json, dict): # unique example
            data = pd.DataFrame(test_json, index=[0])
            original_data = data.copy()
            
        else: # multiple example
            data = pd.DataFrame(test_json, columns=test_json[0].keys())
            original_data = data.copy()
            
        # Instantiate class
        pipeline = Churn()
        
        # data preparation
        data = pipeline.data_preparation(data)
        
        # prediction
        data_prediction = pipeline.get_prediction(model, original_data, data)
        
        return data_prediction
        
        
    else:
        return Reponse('{}', status=200, mimetype='application/json')

if __name__ == '__main__':
    app.run('0.0.0.0')

ModuleNotFoundError: No module named 'churn'

## API Tester

In [105]:
import pickle
import pandas as pd
#from churn.Churn import Churn
from flask import Flask
import requests
import inflection
import response

In [106]:
# loading test dataset
df1 = pd.read_csv("/Users/brunoschirmer/Repos/churn/data/raw/churn.csv")
df1.isna().sum()

RowNumber          0
CustomerId         0
Surname            0
CreditScore        0
Geography          0
Gender             0
Age                0
Tenure             0
Balance            0
NumOfProducts      0
HasCrCard          0
IsActiveMember     0
EstimatedSalary    0
Exited             0
dtype: int64

In [107]:
# convert Dataframe to json
data = json.dumps( df1.to_dict( orient='records' ) )

In [1]:
#data

In [109]:
url = 'http://0.0.0.0:5000/churn/predict'
header = {'Content-type': 'application/json'}

r = requests.post(url=url, data=data, headers=header)

In [110]:
r.status_code

200

In [113]:
d1 = pd.DataFrame( r.json(), columns=r.json()[0].keys())
d1

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,prediction
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,9996,15606229,Obijiaku,771,France,Male,39,5,0.00,2,1,0,96270.64,0,0
9996,9997,15569892,Johnstone,516,France,Male,35,10,57369.61,1,1,1,101699.77,0,0
9997,9998,15584532,Liu,709,France,Female,36,7,0.00,1,0,1,42085.58,1,0
9998,9999,15682355,Sabbatini,772,Germany,Male,42,3,75075.31,2,1,0,92888.52,1,0
